In [10]:
import time
import pyrealsense2 as rs
import numpy as np
import cv2

In [17]:
serial_number = 'f1230450'

# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_device(serial_number)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 1024, 768, rs.format.z16, 30)
# Align depth frame to color frame
align = rs.align(rs.stream.color)

def start_pipeline():
    # Start streaming
    pipeline.start(config)

def stop_pipeline():
    # Stop streaming
    pipeline.stop()

def capture_images(pipeline, position):
    # The pipeline is assumed to already be started
    time_0 = time.time()

    # Capture one frame
    frames = pipeline.wait_for_frames()

    time_1 = time.time()
    print((time_1 - time_0)*1000)


    aligned_frames = align.process(frames)

    # Get aligned depth and color frames
    depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()

    time_2 = time.time()
    print((time_2 - time_0)*1000)

    # Convert depth and color frames to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    time_3 = time.time()
    print((time_3 - time_0)*1000)

    # Save images
    cv2.imwrite(f'./data/rgb_{position}.jpeg', color_image)
    cv2.imwrite(f'./data/depth_{position}.jpeg', depth_image)

    time_4 = time.time()
    print((time_4 - time_0)*1000)


In [4]:
start_pipeline()

# Capture a test frame
for i in range(2):
    frames = pipeline.wait_for_frames()
    frames = pipeline.wait_for_frames()



for i in range(10):  # capture 10 images
    capture_images(pipeline, i)

stop_pipeline()


33.0357551574707
53.68208885192871
53.803443908691406
126.48487091064453
0.019550323486328125
8.107185363769531
8.164644241333008
35.33792495727539
0.010728836059570312
6.911754608154297
6.962299346923828
34.002065658569336
0.009775161743164062
7.257223129272461
7.306814193725586
34.57331657409668
0.017881393432617188
6.679534912109375
6.737709045410156
34.449100494384766
0.014066696166992188
6.95037841796875
6.997108459472656
35.650014877319336
0.011682510375976562
6.580591201782227
6.626129150390625
33.97870063781738
0.01049041748046875
6.628274917602539
6.668329238891602
33.93387794494629
0.01049041748046875
6.660223007202148
6.725311279296875
34.49368476867676
0.010967254638671875
6.705284118652344
6.752729415893555
33.47516059875488


In [3]:
# Initialize the global variable
current_position = 0.0

serial_number = 'f1230450'


def start_pipeline():
    # Start streaming
    pipeline.start(config)

def stop_pipeline():
    # Stop streaming
    pipeline.stop()
    
    
# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_device(serial_number)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 1024, 768, rs.format.z16, 30)
# Align depth frame to color frame
align = rs.align(rs.stream.color)




# Start pipeline and capture initial frames
start_pipeline()

In [6]:
import pyrealsense2 as rs
import json

# Initialize the global variable
current_position = 0.0
serial_number = 'f1230450'

# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_device(serial_number)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 1024, 768, rs.format.z16, 30)

# Align depth frame to color frame
align = rs.align(rs.stream.color)

def start_pipeline():
    # Start streaming
    pipeline.start(config)

def save_intrinsics():
    # Get the intrinsics
    profile = pipeline.get_active_profile()
    depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.color))
    depth_intrinsics = depth_profile.get_intrinsics()

    # Save the intrinsics in the required format
    intrinsics_dict = {
        "K": [
            [depth_intrinsics.fx, 0, depth_intrinsics.ppx],
            [0, depth_intrinsics.fy, depth_intrinsics.ppy],
            [0, 0, 1]
        ],
        # Assuming that the distortion model is "Brown-Conrady",
        # you can get the distortion parameters like this.
        "dist": depth_intrinsics.coeffs,
        "height": depth_intrinsics.height,
        "width": depth_intrinsics.width
    }

    # Write the data to a .txt file
    with open('kd_intrinsics.txt', 'w') as outfile:
        json.dump(intrinsics_dict, outfile, indent=4)

def stop_pipeline():
    # Stop streaming
    pipeline.stop()

# Start pipeline and capture initial frames
start_pipeline()

# Save intrinsics
save_intrinsics()

# Remember to stop the pipeline when you're done
stop_pipeline()


In [7]:
import json
import numpy as np

# Load the data from the .txt file
with open('kd_intrinsics.txt', 'r') as infile:
    intrinsics_dict = json.load(infile)

# Extract the camera matrix 'K' and distortion coefficients 'dist'
K = np.array(intrinsics_dict['K'])
dist = np.array(intrinsics_dict['dist'])

# Now, you can perform various operations with these matrices.
# For example, you might compute the inverse of the camera matrix:
K_inv = np.linalg.inv(K)

# Print the result
print('K:\n', K)
print('dist:\n', dist)
print('K_inv:\n', K_inv)


K:
 [[898.3460083    0.         656.72698975]
 [  0.         898.90014648 367.87283325]
 [  0.           0.           1.        ]]
dist:
 [ 1.34920210e-01 -4.49038595e-01  2.56409607e-04  1.69052277e-04
  4.01904196e-01]
K_inv:
 [[ 0.00111316  0.         -0.73104014]
 [ 0.          0.00111247 -0.40924772]
 [ 0.          0.          1.        ]]


### D435 IR image

In [4]:
import time
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import json
from datetime import datetime
# Initialize the global variable
current_position = 0.0

# serial_number = 'f1230450' #L515
serial_number = '017322071325' #D435
# serial_number = '128422272123'  #D405

save_fold_p = './data/'+serial_number+'_'

now = datetime.now()
dt_string = now.strftime("%Y%m%d%H%M%S")
save_fold_p = save_fold_p + dt_string + '/'

os.makedirs(save_fold_p, exist_ok=True) 


def start_pipeline():
    # Start streaming
    pipeline.start(config)


def stop_pipeline():
    # Stop streaming
    pipeline.stop()
    

def save_intrinsics():
    # Get the intrinsics
    profile = pipeline.get_active_profile()
    
    # Get the depth sensor and set the visual preset
    depth_sensor = profile.get_device().first_depth_sensor()
    depth_scale = depth_sensor.get_depth_scale()
    depth_sensor.set_option(rs.option.visual_preset, 4)  #4 High accuracy for D435 and D405 # 5 L515 short range
    
    # depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.color))  #change it back if it doesnot work
    depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
    depth_intrinsics = depth_profile.get_intrinsics()

    # Save the intrinsics in the required format
    intrinsics_dict = {
        "K": [
            [depth_intrinsics.fx, 0, depth_intrinsics.ppx],
            [0, depth_intrinsics.fy, depth_intrinsics.ppy],
            [0, 0, 1]
        ],
        # Assuming that the distortion model is "Brown-Conrady",
        # you can get the distortion parameters like this.
        "dist": depth_intrinsics.coeffs,
        "height": depth_intrinsics.height,
        "width": depth_intrinsics.width
    }

    # Write the data to a .txt file
    with open(save_fold_p+'kd_intrinsics.txt', 'w') as outfile:
        json.dump(intrinsics_dict, outfile, indent=4)
        
        
    #remove below code after intrinsics finalized
    depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.color)) 
    depth_intrinsics = depth_profile.get_intrinsics()

    # Save the intrinsics in the required format
    intrinsics_dict = {
        "K": [
            [depth_intrinsics.fx, 0, depth_intrinsics.ppx],
            [0, depth_intrinsics.fy, depth_intrinsics.ppy],
            [0, 0, 1]
        ],
        # Assuming that the distortion model is "Brown-Conrady",
        # you can get the distortion parameters like this.
        "dist": depth_intrinsics.coeffs,
        "height": depth_intrinsics.height,
        "width": depth_intrinsics.width
    }

    # Write the data to a .txt file
    with open(save_fold_p+'kdc_intrinsics.txt', 'w') as outfile:
        json.dump(intrinsics_dict, outfile, indent=4)


all_imgs = []

def capture_images(pipeline, position):
    # The pipeline is assumed to already be started
    # Capture one frame
    frames = pipeline.wait_for_frames()

    # Get aligned depth and color frames
    aligned_frames = align.process(frames)

    depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()
    ir_frame = aligned_frames.first(rs.stream.infrared)

    # Convert depth and color frames to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())
    ir_image = np.asanyarray(ir_frame.get_data())

    # append the images and their names into the list
    # all_imgs.append((color_image.copy(), save_fold_p+'rgb_'+str(position)+'.png'))
    cv2.imwrite(save_fold_p + 'rgb_' + str(position) + '.png', color_image)
    cv2.imwrite(save_fold_p + 'depth_' + str(position) + '.png', depth_image)
    cv2.imwrite(save_fold_p + 'ir_' + str(position) + '.png', ir_image)

    # all_imgs.append((depth_image.copy(), save_fold_p+'depth_'+str(position)+'.png'))


# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_device(serial_number)
# config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30) #L515
# config.enable_stream(rs.stream.depth, 1024, 768, rs.format.z16, 30)  #L515
config.enable_stream(rs.stream.color, 1920, 1080, rs.format.bgr8, 30) #D435
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)  #D435
config.enable_stream(rs.stream.infrared, 1, 1280, 720, rs.format.y8, 30)  # Enable IR stream
# config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30) #D405
# config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)  #D405

# Align depth frame to color frame
align = rs.align(rs.stream.color)




# Start pipeline and capture initial frames
start_pipeline()

# Save intrinsics
save_intrinsics()

for i in range(2):
    frames = pipeline.wait_for_frames()
    frames = pipeline.wait_for_frames()

capture_images(pipeline,0)
# print()



# start_time = time.time()
# # time.sleep(3)
# duration = 20  # Duration in seconds

# while True:
#     current_time = time.time()
#     elapsed_time = current_time - start_time
#     # time.sleep(0.4)
#     # Do something here
# capture_images(pipeline, elapsed_time)
    # # time.sleep(0.015)
    # print('Image captured at', elapsed_time)
    # print(current_time)
    # print()
    


    # if elapsed_time > duration:
    #     print('All images captured, now saving')
    #     # Save all remaining images using opencv
    #     for img, name in all_imgs:
    #         cv2.imwrite(name, img)
    #     break


### Creating rotating PCD

In [1]:
from datetime import datetime

import time
import pyrealsense2 as rs
import numpy as np
import cv2
import threading
import json
import os

In [2]:
import pyrealsense2 as rs

# Create a context object. This object owns the handles to all connected realsense devices
pipeline = rs.pipeline()
pipeline.start()

# Get the device
device = pipeline.get_active_profile().get_device()

# Print the serial number
print("Serial number of the device: ", device.get_info(rs.camera_info.serial_number))

# Remember to stop the pipeline once you are done
pipeline.stop()


Serial number of the device:  128422272123


In [18]:
import pyrealsense2 as rs

# serial_number = 'f1230450' #L515
# serial_number = '017322071325' #D435
serial_number = '128422272123'  #D405

ctx = rs.context()
device = None
for dev in ctx.query_devices():
    if dev.get_info(rs.camera_info.serial_number) == serial_number:
        device = dev
        break

if not device:
    print("No device found with the serial number", serial_number)
else:
    best_depth_profile = None
    best_color_profile = None

    # Iterate through all sensors and stream profiles
    for sensor in device.sensors:
        for profile in sensor.get_stream_profiles():
            # Check if this is a depth stream
            if profile.stream_type() == rs.stream.depth:
                # print("Depth stream:", profile.as_video_stream_profile().width(), "x", profile.as_video_stream_profile().height(),profile.fps(), profile.format())
                if best_depth_profile is None or \
                   (profile.as_video_stream_profile().width() * profile.as_video_stream_profile().height() > best_depth_profile.as_video_stream_profile().width() * best_depth_profile.as_video_stream_profile().height() and \
                    profile.fps() > best_depth_profile.fps()):
                    best_depth_profile = profile
            # Check if this is a color stream
            elif profile.stream_type() == rs.stream.color and profile.format() == rs.format.bgr8:
                # print("Color stream:", profile.as_video_stream_profile().width(), "x", profile.as_video_stream_profile().height(),profile.fps(), profile.format())
                if best_color_profile is None or \
                   (profile.as_video_stream_profile().width() * profile.as_video_stream_profile().height() > best_color_profile.as_video_stream_profile().width() * best_color_profile.as_video_stream_profile().height() and \
                    profile.fps() > best_color_profile.fps()):
                    best_color_profile = profile

    if best_depth_profile is not None:
        print("Best depth stream:", best_depth_profile.as_video_stream_profile().width(), "x", best_depth_profile.as_video_stream_profile().height(),
              best_depth_profile.fps(), best_depth_profile.format())

    if best_color_profile is not None:
        print("Best color stream:", best_color_profile.as_video_stream_profile().width(), "x", best_color_profile.as_video_stream_profile().height(),
              best_color_profile.fps(), best_color_profile.format())


Best depth stream: 1280 x 720 30 format.z16
Best color stream: 1280 x 720 30 format.bgr8


In [13]:
# Initialize the global variable
current_position = 0.0

# serial_number = 'f1230450' #L515
# serial_number = '017322071325' #D435
serial_number = '128422272123'  #D405

save_fold_p = './data/test_plant_'

now = datetime.now()
dt_string = now.strftime("%Y%m%d%H%M%S")
save_fold_p = save_fold_p + dt_string + '/'

os.makedirs(save_fold_p, exist_ok=True) 

def start_pipeline():
    # Start streaming
    pipeline.start(config)


def stop_pipeline():
    # Stop streaming
    pipeline.stop()
    

def save_intrinsics():
    # Get the intrinsics
    profile = pipeline.get_active_profile()
    
    # Get the depth sensor and set the visual preset
    depth_sensor = profile.get_device().first_depth_sensor()
    depth_scale = depth_sensor.get_depth_scale()
    depth_sensor.set_option(rs.option.visual_preset, 4)  #4 High accuracy for D435 and D405 # 5 L515 short range
    
    # depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.color))  #change it back if it doesnot work
    depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
    depth_intrinsics = depth_profile.get_intrinsics()

    # Save the intrinsics in the required format
    intrinsics_dict = {
        "K": [
            [depth_intrinsics.fx, 0, depth_intrinsics.ppx],
            [0, depth_intrinsics.fy, depth_intrinsics.ppy],
            [0, 0, 1]
        ],
        # Assuming that the distortion model is "Brown-Conrady",
        # you can get the distortion parameters like this.
        "dist": depth_intrinsics.coeffs,
        "height": depth_intrinsics.height,
        "width": depth_intrinsics.width
    }

    # Write the data to a .txt file
    with open(save_fold_p+'kd_intrinsics.txt', 'w') as outfile:
        json.dump(intrinsics_dict, outfile, indent=4)
        
        
    #remove below code after intrinsics finalized
    depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.color)) 
    depth_intrinsics = depth_profile.get_intrinsics()

    # Save the intrinsics in the required format
    intrinsics_dict = {
        "K": [
            [depth_intrinsics.fx, 0, depth_intrinsics.ppx],
            [0, depth_intrinsics.fy, depth_intrinsics.ppy],
            [0, 0, 1]
        ],
        # Assuming that the distortion model is "Brown-Conrady",
        # you can get the distortion parameters like this.
        "dist": depth_intrinsics.coeffs,
        "height": depth_intrinsics.height,
        "width": depth_intrinsics.width
    }

    # Write the data to a .txt file
    with open(save_fold_p+'kdc_intrinsics.txt', 'w') as outfile:
        json.dump(intrinsics_dict, outfile, indent=4)

all_imgs = []

def capture_images(pipeline, position):
    # The pipeline is assumed to already be started
    time_0 = time.time()

    # Capture one frame
    frames = pipeline.wait_for_frames()

    time_1 = time.time()
    print((time_1 - time_0)*1000)


    aligned_frames = align.process(frames)

    # Get aligned depth and color frames
    depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()

    time_2 = time.time()
    print((time_2 - time_0)*1000)

    # Convert depth and color frames to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    time_3 = time.time()
    print((time_3 - time_0)*1000)

    # Save images
    position = int(position * 10**6)

    # append the images and their names into the list
    position = int(position * 10**6)
    all_imgs.append((color_image.copy(), save_fold_p+'rgb_'+str(position)+'.png'))
    cv2.imwrite(save_fold_p+'rgb_'+str(position)+'.png' , color_image)
    cv2.imwrite(save_fold_p+'depth_'+str(position)+'.png', depth_image)
    # all_imgs.append((depth_image.copy(), save_fold_p+'depth_'+str(position)+'.png'))

    time_4 = time.time()
    print((time_4 - time_0)*1000)



 

In [11]:
# Configure the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_device(serial_number)
# config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30) #L515
# config.enable_stream(rs.stream.depth, 1024, 768, rs.format.z16, 30)  #L515
# config.enable_stream(rs.stream.color, 1920, 1080, rs.format.bgr8, 30) #D435
# config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)  #D435
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30) #D405
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)  #D405

# Align depth frame to color frame
align = rs.align(rs.stream.color)




# Start pipeline and capture initial frames
start_pipeline()

# Save intrinsics
save_intrinsics()

for i in range(2):
    frames = pipeline.wait_for_frames()
    frames = pipeline.wait_for_frames()

capture_images(pipeline, 0)
print()



start_time = time.time()
# time.sleep(3)
duration = 20  # Duration in seconds

while True:
    current_time = time.time()
    elapsed_time = current_time - start_time
    # time.sleep(0.4)
    # Do something here
    capture_images(pipeline, elapsed_time)
    # time.sleep(0.015)
    print('Image captured at', elapsed_time)
    print(current_time)
    print()
    


    if elapsed_time > duration:
        print('All images captured, now saving')
        # Save all remaining images using opencv
        for img, name in all_imgs:
            cv2.imwrite(name, img)
        break

33.41984748840332
54.711103439331055
55.716514587402344
175.7512092590332

0.0
11.632204055786133
11.632204055786133
63.42959403991699
Image captured at 0.0
1691459939.41803

0.0
9.104013442993164
9.104013442993164
72.20721244812012
Image captured at 0.06342959403991699
1691459939.4814596

0.0
11.906147003173828
11.906147003173828
74.70560073852539
Image captured at 0.13663840293884277
1691459939.5546684

0.0
11.686086654663086
11.686086654663086
65.85001945495605
Image captured at 0.21134400367736816
1691459939.629374

0.0
10.266780853271484
10.266780853271484
60.90068817138672
Image captured at 0.2771940231323242
1691459939.695224

0.0
10.887384414672852
10.887384414672852
63.08579444885254
Image captured at 0.33809471130371094
1691459939.7561247

0.0
10.000467300415039
10.000467300415039
64.3317699432373
Image captured at 0.4011805057525635
1691459939.8192105

0.0
11.346817016601562
11.346817016601562
62.270164489746094
Image captured at 0.4655122756958008
1691459939.8835423

0.0
8.